In [1]:
import numpy as np
import pandas as pd
from visits import Visit, Cohort, MissingDataError, holm_bonferroni
import glob
import pymannkendall as mk
from datetime import datetime, timedelta

In [2]:
cohort_1_details = pd.read_csv("cohort_1_details_v5.csv", sep=',')
cohort_2_details = pd.read_csv("cohort_2_details_v5.csv", sep=',')
cohort_3_details = pd.read_csv("cohort_3_details_v5.csv", sep=',')
cohort_4_details = pd.read_csv("cohort_4_details_v5.csv", sep=',')

In [3]:
cohort_3_details

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,intubation_flag,extubation_flag,extubation_deid_date,extubation_score,re_intubation_deid_date,icu_ward,total_visits,extubation_duration,age (days)
0,49,OL0043,2000-07-12 00:00:00,2018-04-25 00:00:00,Female,2,2016-06-08 18:56:00,2016-06-09 13:01:00,NaN,NaN,1,1.0,2016-06-09 02:00:00,5.0,NaN,PICU,2,1320.0,5811.083333
1,91,JT0073,2003-04-26 00:00:00,2018-08-18 00:00:00,Male,1,2018-07-27 12:50:00,2018-07-30 15:15:00,NaN,NaN,1,1.0,2018-07-29 15:15:00,5.0,NaN,FLAMI,1,525.0,5573.635417
2,124,CD0102,2003-02-18 00:00:00,2018-09-17 00:00:00,Male,1,2017-08-05 09:37:00,2017-08-12 17:56:00,NaN,NaN,1,1.0,2017-08-09 19:00:00,5.0,NaN,PICU,1,300.0,5286.791667
3,160,XX0133,2002-01-25 00:00:00,2018-01-24 00:00:00,Male,1,2016-02-15 01:15:00,2016-02-26 12:35:00,NaN,NaN,1,1.0,2016-02-21 11:00:00,5.0,NaN,PICU,1,780.0,5140.458333
4,181,DN0147,2000-08-11 00:00:00,2017-08-19 00:00:00,Female,1,2016-10-13 23:45:00,2016-10-15 13:42:00,NaN,NaN,1,1.0,2016-10-14 14:00:00,5.0,NaN,PICU,1,600.0,5908.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,5461,PM3943,2018-09-19 00:00:00,2018-10-20 00:00:00,Male,1,2018-10-14 01:15:00,2018-10-20 23:10:00,NaN,NaN,1,1.0,2018-10-20 23:00:00,5.0,NaN,PICU,1,60.0,31.958333
210,5483,IE3963,2013-04-09 00:00:00,2018-11-20 00:00:00,Female,1,2018-10-24 20:15:00,2018-11-20 18:46:00,NaN,NaN,1,1.0,2018-10-31 13:00:00,5.0,NaN,FLAMI,1,660.0,2031.541667
211,5535,BB4010,2003-09-15 00:00:00,2018-11-20 00:00:00,Female,1,2018-11-06 02:00:00,2018-11-20 23:25:00,NaN,NaN,1,1.0,2018-11-20 23:00:00,5.0,NaN,FLAMI,1,60.0,5545.958333
212,5551,WE4023,2018-11-10 00:00:00,2019-01-03 00:00:00,Female,2,2018-12-23 16:36:00,2019-01-03 10:20:00,NaN,NaN,1,1.0,2019-01-03 09:30:00,5.0,NaN,NICU,2,870.0,54.395833


In [4]:
# f={"icu_visit":"max", "extubation_deid_date":"head(1)"}
cohort_3_details_max = cohort_3_details.groupby("Project ID", as_index=False)['icu_visit'].agg(max)
cohort_3_details_max

,Project ID,icu_visit
0,AC0922,1
1,AF1531,6
2,AH3026,3
3,AV2830,1
4,AW3455,1
...,...,...
161,YX0518,1
162,ZD2042,1
163,ZI1827,1
164,ZS0567,1


In [5]:
for idx in cohort_3_details.index:
    if (cohort_3_details['Project ID'].loc[idx], cohort_3_details['icu_visit'].loc[idx]) in zip(cohort_3_details_max['Project ID'],cohort_3_details_max['icu_visit']):
        pass
    else:
        cohort_3_details = cohort_3_details.drop(index=idx)

In [6]:
cohort_3_details

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,intubation_flag,extubation_flag,extubation_deid_date,extubation_score,re_intubation_deid_date,icu_ward,total_visits,extubation_duration,age (days)
0,49,OL0043,2000-07-12 00:00:00,2018-04-25 00:00:00,Female,2,2016-06-08 18:56:00,2016-06-09 13:01:00,NaN,NaN,1,1.0,2016-06-09 02:00:00,5.0,NaN,PICU,2,1320.0,5811.083333
1,91,JT0073,2003-04-26 00:00:00,2018-08-18 00:00:00,Male,1,2018-07-27 12:50:00,2018-07-30 15:15:00,NaN,NaN,1,1.0,2018-07-29 15:15:00,5.0,NaN,FLAMI,1,525.0,5573.635417
2,124,CD0102,2003-02-18 00:00:00,2018-09-17 00:00:00,Male,1,2017-08-05 09:37:00,2017-08-12 17:56:00,NaN,NaN,1,1.0,2017-08-09 19:00:00,5.0,NaN,PICU,1,300.0,5286.791667
3,160,XX0133,2002-01-25 00:00:00,2018-01-24 00:00:00,Male,1,2016-02-15 01:15:00,2016-02-26 12:35:00,NaN,NaN,1,1.0,2016-02-21 11:00:00,5.0,NaN,PICU,1,780.0,5140.458333
4,181,DN0147,2000-08-11 00:00:00,2017-08-19 00:00:00,Female,1,2016-10-13 23:45:00,2016-10-15 13:42:00,NaN,NaN,1,1.0,2016-10-14 14:00:00,5.0,NaN,PICU,1,600.0,5908.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,5461,PM3943,2018-09-19 00:00:00,2018-10-20 00:00:00,Male,1,2018-10-14 01:15:00,2018-10-20 23:10:00,NaN,NaN,1,1.0,2018-10-20 23:00:00,5.0,NaN,PICU,1,60.0,31.958333
210,5483,IE3963,2013-04-09 00:00:00,2018-11-20 00:00:00,Female,1,2018-10-24 20:15:00,2018-11-20 18:46:00,NaN,NaN,1,1.0,2018-10-31 13:00:00,5.0,NaN,FLAMI,1,660.0,2031.541667
211,5535,BB4010,2003-09-15 00:00:00,2018-11-20 00:00:00,Female,1,2018-11-06 02:00:00,2018-11-20 23:25:00,NaN,NaN,1,1.0,2018-11-20 23:00:00,5.0,NaN,FLAMI,1,60.0,5545.958333
212,5551,WE4023,2018-11-10 00:00:00,2019-01-03 00:00:00,Female,2,2018-12-23 16:36:00,2019-01-03 10:20:00,NaN,NaN,1,1.0,2019-01-03 09:30:00,5.0,NaN,NICU,2,870.0,54.395833


In [7]:
# cohort_3_details = cohort_3_details[(cohort_3_details['Project ID'], cohort_3_details['icu_visit']) in list(zip(cohort_3_details_max['Project ID'],cohort_3_details_max['icu_visit']))]
# zip(cohort_3_details_max['Project ID'],cohort_3_details_max['icu_visit'])]

In [8]:
path = "../In2Research_data/data/"

all_files = glob.glob(path + "/*")

cohort_dict = {1: [], 2: [], 3: [], 4: [],}

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_1_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 1, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_2_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 2, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_3_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 3, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_4_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 4, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_5_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 5, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_6_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 6, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_7_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 7, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

for letter1 in all_files:
    for letter2 in glob.glob(letter1 + "/*"):
        for id in glob.glob(letter2 + "/*"):
            if any("_8_" in file for file in glob.glob(id + "/*")):
                visit = Visit(id, 8, cohort_1_details, cohort_2_details, cohort_3_details, cohort_4_details)
                if visit.cohort:
                    cohort_dict[visit.cohort].append(visit)

In [9]:
len(cohort_dict[3])

166

In [10]:
cohort3 = Cohort(cohort_dict[3][100:-1], 3)

In [11]:
cohort3.visits

In [12]:
variance_hr_results_c3 = cohort3.var_results(hr = True)

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
WM1952 visit 1 failed: Extubated period is too short to carry out analysis.
WJ3130 visit 1 failed: 'NoneType' object has no attribute 'copy'


There is no heart rate data available during the extubated period.


RU1595 visit 2 failed: 'NoneType' object has no attribute 'copy'


There is no heart rate data available during the extubated period.


NW2380 visit 2 failed: 'NoneType' object has no attribute 'copy'


...

There is no heart rate data available during the extubated period.


YO3343 visit 2 failed: Extubated period is too short to carry out analysis.
WE4023 visit 2 failed: 'NoneType' object has no attribute 'copy'


There is no heart rate data available during the extubated period.


: 

: 